In [ ]:
import asyncio
import threading

### 1. 协程

In [2]:
def consumer():
    r = ""
    while True:
        n = yield r
        if not n:
            return
        print(f"[CONSUMER] Consuming {n}...")
        r = "200 OK"


def produce(c):
    c.send(None)
    n = 0
    while n<5:
        n = n+1
        print(f"[PRODUCER] Producing {n}...")
        r = c.send(c)
        print(f"[PRODUCER] Consumer return: {r}")
    c.close()

In [3]:
c = consumer()
produce(c)

[PRODUCER] Producing 1...
[CONSUMER] Consuming <generator object consumer at 0x0000027E9D6E92A0>...
[PRODUCER] Consumer return: 200 OK
[PRODUCER] Producing 2...
[CONSUMER] Consuming <generator object consumer at 0x0000027E9D6E92A0>...
[PRODUCER] Consumer return: 200 OK
[PRODUCER] Producing 3...
[CONSUMER] Consuming <generator object consumer at 0x0000027E9D6E92A0>...
[PRODUCER] Consumer return: 200 OK
[PRODUCER] Producing 4...
[CONSUMER] Consuming <generator object consumer at 0x0000027E9D6E92A0>...
[PRODUCER] Consumer return: 200 OK
[PRODUCER] Producing 5...
[CONSUMER] Consuming <generator object consumer at 0x0000027E9D6E92A0>...
[PRODUCER] Consumer return: 200 OK


### 2. asyncio - 异步编程

In [4]:
async def hello():
    print("Hello world!")
    # 异步调用asyncio.sleep(1):
    await asyncio.sleep(1)
    print("Hello again!")

asyncio.run(hello())

RuntimeError: asyncio.run() cannot be called from a running event loop

执行结果如下：
```python
Hello!
(等待约1秒)
Hello again!
```

In [ ]:
# 传入name参数:
async def hello(name):
    # 打印name和当前线程:
    print("Hello %s! (%s)" % (name, threading.current_thread))
    # 异步调用asyncio.sleep(1):
    await asyncio.sleep(1)
    print("Hello %s again! (%s)" % (name, threading.current_thread))
    return name

async def main():
    L = await asyncio.gather(hello("Bob"), hello("Alice"))
    print(L)

asyncio.run(main())

执行结果如下：
```python
Hello Bob! (<function current_thread at 0x10387d260>)
Hello Alice! (<function current_thread at 0x10387d260>)
(等待约1秒)
Hello Bob again! (<function current_thread at 0x10387d260>)
Hello Alice again! (<function current_thread at 0x10387d260>)
['Bob', 'Alice']
```

In [ ]:
async def wget(host):
    print(f"wget {host}...")
    # 连接80端口:
    reader, writer = await asyncio.open_connection(host, 80)
    # 发送HTTP请求:
    header = f"GET / HTTP/1.0\r\nHost: {host}\r\n\r\n"
    writer.write(header.encode("utf-8"))
    await writer.drain()

    # 读取HTTP响应:
    while True:
        line = await reader.readline()
        if line == b"\r\n":
            break
        print("%s header > %s" % (host, line.decode("utf-8").rstrip()))
    # Ignore the body, close the socket
    writer.close()
    await writer.wait_closed()
    print(f"Done {host}.")


async def main():
    await asyncio.gather(
        wget("www.sina.com.cn"), wget("www.sohu.com"), wget("www.163.com")
    )


asyncio.run(main())

执行结果如下：

```python
wget www.sohu.com...
wget www.sina.com.cn...
wget www.163.com...
(等待一段时间)
(打印出sohu的header)
www.sohu.com header > HTTP/1.1 200 OK
www.sohu.com header > Content-Type: text/html
...
(打印出sina的header)
www.sina.com.cn header > HTTP/1.1 200 OK
www.sina.com.cn header > Date: Wed, 20 May 2015 04:56:33 GMT
...
(打印出163的header)
www.163.com header > HTTP/1.0 302 Moved Temporarily
www.163.com header > Server: Cdn Cache Server V2.0
...
```